## Langfuse Trace Inspector

This notebook helps you fetch and inspect a single trace object from your Langfuse project. This is useful for debugging and understanding the data structure before building dashboards.

### 2. Imports and Setup

Import the required libraries and load your environment variables from the `.env` file in your project root. This will configure the Langfuse client automatically.

In [1]:
import os
from langfuse import get_client
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Initialize the Langfuse client
try:
    langfuse_client = get_client()
    print("✅ Langfuse client initialized successfully.")
except Exception as e:
    print(f"❌ Failed to initialize Langfuse client: {e}")

✅ Langfuse client initialized successfully.


### 3. Fetch the Latest Trace

We'll first list the most recent traces and then fetch the full details for the very latest one.

In [3]:
latest_trace = None

try:
    # List the 5 most recent traces to find the latest one
    trace_list = langfuse_client.api.trace.list(limit=5)

    if trace_list.data:
        latest_trace_id = trace_list.data[0].id
        print(f"Found latest trace ID: {latest_trace_id}")
        
        # Fetch the full trace object using its ID
        latest_trace = langfuse_client.api.trace.get(latest_trace_id)
        print("✅ Successfully fetched the full trace object.")
    else:
        print("⚠️ No traces found. Please run the agent via 'test_graph.py' to generate a trace.")
except Exception as e:
    print(f"❌ An error occurred while fetching traces: {e}")

Found latest trace ID: 791067a9ef9baaa5c1285caca52b2744
✅ Successfully fetched the full trace object.


### 4. Inspect the Trace Object

Now, let's print the full trace object to see its structure and all available fields.

In [4]:
if latest_trace:
    # Print the entire object
    from pprint import pprint
    pprint(latest_trace)

TraceWithFullDetails(id='791067a9ef9baaa5c1285caca52b2744', timestamp=datetime.datetime(2025, 7, 11, 10, 32, 4, 478000, tzinfo=datetime.timezone.utc), name='LangGraph', input={'messages': [{'content': 'Thanks for letting me know the departure city.  To help you plan your trip, please also tell me your travel dates, desired class of service (e.g., economy, business), and preferred airline.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}, {'content': 'what city I told you', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}], 'detected_language': 'hi-translit', 'original_query': 'what city i told you'}, output=None, session_id=None, release=None, version=None, user_id=None, metadata={'resourceAttributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.versio

In [9]:
print(type(latest_trace))

<class 'langfuse.api.resources.commons.types.trace_with_full_details.TraceWithFullDetails'>


In [13]:
latest_trace_df = pd.DataFrame([vars(latest_trace)])

In [14]:
latest_trace_df

,id,timestamp,name,input,output,session_id,release,version,user_id,metadata,...,html_path,latency,total_cost,observations,scores,updatedAt,bookmarked,createdAt,projectId,externalId
0,791067a9ef9baaa5c1285caca52b2744,2025-07-11 10:32:04.478000+00:00,LangGraph,{'messages': [{'content': 'Thanks for letting ...,None,None,None,None,None,{'resourceAttributes': {'telemetry.sdk.languag...,...,/project/cmcw9nhci004nad06kf6dx7gz/traces/7910...,4.034,0.0,[id='afed2fbae328f0e4' trace_id='791067a9ef9ba...,[],2025-07-11T10:32:14.355Z,False,2025-07-11T10:32:09.000Z,cmcw9nhci004nad06kf6dx7gz,None


In [10]:
latest_trace_df = pd.DataFrame([latest_trace.to_dict()])

AttributeError: 'TraceWithFullDetails' object has no attribute 'to_dict'

### 5. Access Specific Fields

Here's how you can access the key fields that are most useful for your dashboard.

In [4]:
if latest_trace:
    print(f"Trace ID: {latest_trace.id}")
    print(f"Timestamp: {latest_trace.timestamp}")
    print(f"User ID: {latest_trace.user_id}")
    print(f"Number of Observations: {len(latest_trace.observations)}")
    
    print("\n--- Input Object ---")
    # Using pandas to pretty-print the dictionary
    display(pd.Series(latest_trace.input).to_frame('Value'))

    print("\n--- Output Object ---")
    display(pd.Series(latest_trace.output).to_frame('Value'))
    
    print("\n--- Metadata Object ---")
    # Metadata is only present if you add it via trace.update() or similar
    if latest_trace.metadata:
        display(pd.Series(latest_trace.metadata).to_frame('Value'))
    else:
        print("No custom metadata found on this trace.")

Trace ID: 6364dfe204b6da3a5e229fc7c0a02bb9
Timestamp: 2025-07-10 11:32:17.238000+00:00
User ID: None
Number of Observations: 17

--- Input Object ---


,Value
messages,[{'content': 'testing just to see if its worki...
detected_language,en
original_query,testing just to see if its working or not



--- Output Object ---


,Value
original_query,testing just to see if its working or not
detected_language,en
messages,"[{'content': 'Yes, the system is working! Is ..."
intents,[general]
confidence,90
processed_intents,[general]
last_completed_intent,general
output,The system is working. Is there anything else...
aggregated_output,"Yes, the system is working! Is there anything..."



--- Metadata Object ---


,Value
resourceAttributes,"{'telemetry.sdk.language': 'python', 'telemetr..."
scope,"{'name': 'langfuse-sdk', 'version': '3.1.3', '..."


In [6]:
print(latest_trace.output.get("aggregated_output", "N/A"))

Yes, the system is working!  Is there anything else I can assist you with?
